In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install --upgrade --force-reinstall pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 30.4 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six

In [4]:
import pandas as pd

CSV_PATH = '/content/drive/MyDrive/conversation_data.csv'
df = pd.read_csv(CSV_PATH)


In [5]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))


In [8]:
# ------------------ FIXED IMPORTS ------------------
import datetime
import json
import random
import ast
import re
import pandas as pd

from openai import OpenAI
from google.colab import userdata


DEBUG_ROUTING = True
HARD_SWITCH_ON_CATEGORY_CHANGE = True

STAR_DIVIDER = "\n" + "#" * 3 + "\n"

# ---------------------------------------------
# CONFIG + LOAD CSV (unchanged)
# ---------------------------------------------
CSV_PATH = '/content/drive/MyDrive/conversation_data.csv'
df = pd.read_csv(CSV_PATH)

def _to_num(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip().lower()
    if s in {"", "n/a", "na", "null", "none", "error", "err"}:
        return None
    try:
        return float(s)
    except (ValueError, TypeError):
        return None

row = df.iloc[100]
transcript = ast.literal_eval(row["conversation_messages"])
scores_raw = ast.literal_eval(row["mirs_scores"])

metrics = {
    k: {
        "score": _to_num(v.get("score")),
        "explanation": (v.get("explanation") or "").strip()
    }
    for k, v in scores_raw.items()
}
score_dict = metrics

# ---------------------------------------------
# INIT OPENAI
# ---------------------------------------------
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# ========= MIRS DEFINITIONS =========
MIRS_DEFINITIONS = {
   "OPENING": """Introduces self, clarifies role, and inquires how to address patient.
NOTE: Learner begins with the introduction of self, clarification of roles, and inquiry of how to address patient. The opening question identifies the problems or issues that the patient wishes to address.
Example: “Hello, I’m Carol Redding, (shaking patient’s hand) a student working with the team. Would you prefer I call you, Ms. Black or Phyllis? Are you comfortable right now? What brings you in today?”
Score 5: Learner introduces self, clarifies role, and inquires how to address patient.  Uses patient name. Score 3: Learner introduction is missing a critical element(s). Score 1: There is no introduction.""",
   "ELICITS SPECTRUM OF CONCERNS": """Elicits full spectrum of concerns at onset in first few minutes of encounter.
NOTE: Learner elicits the patient's full spectrum of concerns other than those expressed in the chief complaint within the first few minutes of the encounter.
To ensure that the full spectrum of concerns has been obtained, asks if there is anything else the patient wishes to discuss until the patient says, “No.” Making sure in the beginning of the encounter that all issues the patient wishes to address have been elicited helps learners to prioritize and aids in making a time-efficient, patient-centered encounter.
Score 5: Learner elicits the patient’s full spectrum of concerns within the first few minutes. Score 3: Learner elicits some of the patient’s concerns. Score 1: Learner fails to elicit the patient’s concerns.""",
   "NEGOTIATES PRIORITIES & SETS AGENDA": """Purpose, agenda, plan, and patient agreement.
NOTE: Learner ascertains the patient’s concerns. In negotiating priorities, a balance is struck between patient concerns and providers medical understanding of which problems might be more immediately important.
An agenda is negotiated between learner and patient. In agenda setting and negotiating, the patient should not be told what is going to occur, but instead share in making an agreed plan.
Score 5: Learner fully negotiates priorities of patient concerns, listing all concerns and sets the agenda. Agenda is negotiated between learner and patient. Score 3: Learner elicits only partial concerns, therefore does not accomplish the complete patient agenda OR learner set the goals. Score 1: Learner does not negotiate priorities or set agenda.""",
   "ELICITING THE NARRATIVE THREAD": """Gives patient an opportunity to tell the story without interruption at interview onset.
NOTE: After the agenda is set, the learner encourages the patient to talk about the problem(s), in their own words and listens attentively without interrupting (except for encouragement to continue) until the patient has finished talking about the problem(s).
This technique is an effective tool for data gathering, time-efficiency, and rapport building. It allows the patient to tell their story and can help learners gain a large amount of needed data in a short amount of time.
Example: “Tell me what’s been going on from when your symptoms first started up to today.”
Score 5: Learner encourages and lets the patient talk about the problem. Learner does not stop the patient or introduce new information. Score 3: Learner begins to let the patient talk about their problem but either interrupts with focused questions or introduces new information into the conversation. Score 1: Learner fails to let the patient tell their story/talk about their problem OR sets the pace with a Q & A style, not a conversational style.""",
   "TIMELINE": """Chronological progression of all of the symptoms from onset to present time.
NOTE: The timeline pertains to the information contained in the chief complaint and history of the patient's current illness. To obtain a timeline, Learner should inquire when the patient was last free of this problem, and then follow the progression of the first signs and symptoms to the present.
By carefully following the chronological progression of the complaint, the Learner will avoid missing important information.
If several symptoms are reported, it is important that their chronological relationship to each other be determined. Learners need not gather the information in a chronological order or all at once if the information needed is obtained during the encounter.
Example: A 56-year-old male presents with left side chest pain for two hours. The patient's chest pain first occurred two years ago but only upon exertion and disappeared after a few minutes. One year ago, the pain increased in intensity and was diagnosed as angina pectoris. Nifedipine (l0 mg) qid was taken and the pain stopped occurring one month later.
The patient continued to take Nifedipine (l0) bid and is still currently. Two hours ago, he experienced chest pain on the left and one hour ago he experienced sweating, faintness, palpitations, and the pain radiated to the left shoulder.
Score 5: Learner obtains sufficient information so that a chronology of the chief complaint and history of the present illness can be established. The chronology of any associated symptoms is also established. Score 3: Learner obtains some of the information necessary to establish a chronology. Learner may fail to establish a chronology for any associated symptoms. Score 1: Learner fails to obtain information necessary to establish a chronology.""",
   "TYPES OF QUESTIONS": """Begins with open-ended question (describe, tell me about), followed by direct questions; avoids leading, negative, and multiple questions.
NOTE: Learners should follow a line of inquiry that progresses from open-ended to specific followed with specific questions.
Open-ended Questions (or open-ended prompts) allow the learner to obtain a large amount of information about a particular area. It allows the patient to tell their story. This should be used to begin a line of inquiry then follow up with more focused and direct questions. Example: "What brings you here today?" or "Tell me about your general health.”
Direct or Specific Questions are used to gather specific pertinent information. Example: “How old were you when you had your tonsils removed?" or "When did your pain begin?" or "How long have you had the pain?"
Other types of direct questions elicit a "yes" or "no" answer from the patient, or a response to a choice that the learner has provided. Example various types of questions:
Learner (L): "Tell me about your problem." (Open-ended) Patient (P): "For two weeks, I've had a constant pain in my stomach, right here (points), above my navel." L: "Tell me about the pain." (Open-ended) P: "It's a burning sensation." L: "Is it a deep pain?" (Direct) P: "Yes, a deep one." L: "Does the pain move around?" (Direct) P: "No."  L: "What makes the pain feel worse?" (Open-ended)
Learners should avoid using direct or (particularly) forced choice questions in beginning a line of inquiry because it restricts the possible flow of information and makes obtaining the necessary information a tedious task. Example: if NOT beginning with an open-ended question such as: "Tell me about the pain.” they must ask several direct questions: L: "Is the pain an ache?" P: "No."  L: "Is it a stabbing pain?" P: "No."  L: "Is it a dull pain?" P: "No."
Learners should avoid these poor question types:
Leading Questions supply a particular answer for the patient, desired answer is implied by how the question is phrased. This should also be avoided because some patients may agree with the leading questions rather than contradicting the learner. Example: "No headaches? Right?"
"Why" Questions put the patient on the defensive and should be avoided.
Example: "Why didn’t you come in sooner, you've had the problem for six weeks?"
Multiple Questions are a series of short questions asked in succession without allowing the patient to answer each individually. The patient may be confused about which question to answer.
Example: “Does the pain feel like it’s as sharp after dinner or is it different than before dinner? Multiple questions may also be one question listing many options. Example: "Has anyone in your family ever had cancer, diabetes, heart disease, or high blood pressure?"
Score 5: Learner begins with an open-ended question and follows up with specific or direct questions. Each major line of questioning starts with an open-ended question. No poor question types are used. Score 3: Learner often fails to begin with open-ended questions but rather employs specific or direct questions to gather information OR uses a few leading, why or multiple questions. Score 1: Learner asks many why questions, multiple questions, or leading questions.""",
   "VERIFICATION OF PATIENT INFORMATION": """Pursue/verify details of symptoms, events, meds (dates, dosages, quantities.)
NOTE: In the interest of gaining as accurate information as possible, the learner must verify and clarify the information given by the patient.
Clarifying: “Can you explain what you mean by ‘weak.’?”
Verification: “I’m confused, you said you’d never been short of breath before but sounds like you had a similar feeling last year. Can you clear that up for me?”
If responses from the patient include specific diagnoses or medications, it is the task of the Learner to ascertain if the patient knows how the diagnosis was made or determine the quantity of medication.
Example: “You said you were allergic to penicillin. How do you know that?”
Score 5: Learner always seeks clarification, verification, and specificity of the patient’s responses. Score 3: Learner sometimes seeks clarification, verification, and specificity of the patient’s responses (but not always.) Score 1: Learner fails to clarify or verify patient’s responses, accepting information at face value.""",
   "USE OF JARGON": """Lay vocabulary is used; medical terms are explained immediately.
NOTE: One of the skills needed is the ability to communicate with the patient with terms known to lay persons – to substitute jargon or difficult medical terms. Learners may make erroneous assumptions about the patient's level of sophistication based on one or two medical terms used by the patient.
Jargon may also be misleading to a patient who does not want to admit they don’t understand the question/term. Therefore, learners should define questionable terms. Additionally, learners should be aware of educational levels. By keeping these things in mind when communicating with a patient, information will be clearer and long-term compliance easier to obtain.
Example: “Was it a productive cough?" followed by “Productive means does anything come up when you cough?”
Score 5: Learner asks questions and provides information in language which is easily understood. Content is free of difficult medical terms and jargon. Words are immediately defined. Language used is appropriate to patient level of education. Score 3: Learner occasionally uses medical jargon, failing to define medical terms unless specifically requested to do so by the patient. Score 1: Learner uses difficult medical terms and jargon throughout the conversation.""",
   "PATIENT'S PERSPECTIVE (BELIEFS)": """Patient is asked about perception of problems/issues.
NOTE: Learner should elicit the patient's perspective on symptoms in order for it to be effectively diagnosed and treated. The patient's beliefs about the beginning of their illness may affect their ability to talk about their symptoms or to understand the diagnosis. Gaining the patient perspective uncovers hidden concerns.
One method of eliciting beliefs is to encourage the patient to discuss FIFE:
Feelings: addresses the patient’s feelings about each of the problems.
Ideas: determines and acknowledges patient’s ideas (belief of cause) for each of the problems.
Function: determines how each problem affects the patient’s life.
Expectations: determines patient’s goals, what help the patient had expected for each problem.
Does NOT have to be scripted, asked at one time or in order. It can occur at any point in the conversation. Ideas asked first may elicit feelings. Expectations may be incorporated into the shared agenda at the encounter onset. Function may help elicit information in the HPI.
Example: L: "What do you think is going on?  P: I’m worried [Feelings] I may have something serious [Idea].  L: Why do you think it is serious?  P: My uncle died of stomach cancer one year ago [Hidden concern].
L: I’m sorry to hear about your uncle. I understand your concern. How has this problem been affecting you at work and home? [Function]. P: Aside from the pain, I’m preoccupied with wondering what is making me sick.
L: Besides wanting to find out the cause of this problem, what can I do to help? [Expectations].”
Score 5: Learner elicits the patient’s healing practices and perspectives on their illness, including their beliefs about its beginning, Feelings, Ideas of cause, Function and Expectations (FIFE). Score 3: Learner elicits some of the patient’s perspective on their illness AND/OR Learner does not follow through with addressing beliefs. Score 1: Learner fails to elicit the patient’s perspective.""",
   "IMPACT OF ILLNESS ON PATIENT & SELF-IMAGE": """Explores impact of illness on activities of daily living, finances/work, and social function; explores feelings about illness.
NOTE: Learner must address the impact on self-image that certain illnesses/symptoms/situations may have on the patient. They should explore these issues in depth to the satisfaction of the patient. Learner also offers counseling or recommends resources, if appropriate.
Example: Immediately after a heart attack, a patient needs to change their sexual and physical activity, this may impact the way they view themselves. L: “Some patients stop exercising after having a heart attack for fear of triggering another. How is it affecting you?” Tell me about your exercise routine now? ... I can suggest a specialist to help.”
Score 5: Learner inquires about the patient’s feelings about their illness, how it has changed their life and explores these issues. Learner offers counseling or resources to help. Score 3: Learner partially addresses the impact of symptoms on patient’s life or self-image AND/OR Learner offers no counseling or resources to help. Score 1: Learner fails to acknowledge any impact of the illness on the patient’s life or self-image.""",
   "IMPACT OF ILLNESS ON FAMILY": """Explores impact of illness/treatment on family members.
NOTE: Depending on the diagnosis, as well as the information obtained during the personal history, there could be a tremendous impact of the patient's illness on the family and the family's lifestyle.
Example: For a patient with cancer this would certainly affect family members and family lifestyle because of the need for frequent treatment, side effects of drugs, potentially decreased family income, etc.
The Learner must address this issue and explore it in depth to the patient’s satisfaction.
Example: L: “You told me your child cries all through all night. Who else at home is affected by this?”
P: “My husband. He cannot sleep and has missed work.”  L: “OK, let’s discuss ways to relieve this stress ...”
Score 5: Learner inquires about the structure of the patient’s family. Learner addresses the impact of the patient’s illness and/or treatment on family. Learner explores these issues. Score 3: Learner recognizes the impact of the illness and/or treatment on the family members and family lifestyle but fails to explore these issues adequately. Score 1: Learner fails to address the impact of the illness and/or treatment on the family members and on family lifestyle.""",
   "SUPPORT SYSTEMS": """Inquires about friends, family, social services, support groups, finances, and spiritual resources.
NOTE: Learner should explore the patient’s means of financial, social, and emotional support. Support systems might include other family members, friends, and the patient’s workplace. These are current resources which could be employed immediately. Learners may suggest other community resources including charitable organizations, self-help groups, etc., not yet thought of or known to the patient.
Examples: “These tests can be expensive; are you concerned about this?” “It sounds like you’ve been through a difficult time. Do you have someone you can talk to?” “Is there anyone that can help with the children until you’re feeling better?” “You told me that you are involved in your church, could they be a source of help?”
Score 5: Learner determines what emotional/financial support patient has, learns about health care access. Inquires about other resources available to patient/family, suggests appropriate community resources. Score 3: Learner determines some of the available support. Score 1: Learner fails to determine what support is currently available to the patient.""",
   "VERBAL FACILITATION SKILLS & ENCOURAGEMENT": """Verbally encourage patient to tell the story; verbal reinforcement for positive behaviors.
NOTE: It is important to actively encourage patients to continue their storytelling. Any behavior that has the effect of inviting patients to say more about the area being discussing is a facilitative response. Learner follows up patient’s initial story with focused facilitation skills to broaden and complete the story. The use of short statements and echoing (using a few words of the patient's last sentence) encourage the patient to say more about a topic and indicate that a learner is interested in what the patient is saying and wants them to continue. Learners use verbal encouragement to motivate the patient toward a cooperative relationship and continued health care throughout the encounter e.g., responding to the patient in a way that they feel encouraged in starting/continuing proper health care techniques.
Verbal Encouragement and use of occasional social praises.
Example: L: “Quitting smoking is excellent, that takes willpower!"
Short statements encourage them to continue talking about the problem.
Example “Go on… tell me more,”
Echoing to encourage patients to elaborate on a topic.
Example: P: “I couldn’t take in a good breath.”  L: “Did you feel as if you couldn’t get your breath?”
Score 5: Learner uses facilitative skills throughout the conversation. Verbal encouragement, use of short statements, and echoing used regularly when appropriate. Learner provides the patient with intermittent verbal encouragement. Score 3: Learner uses some facilitative skills but not consistently or at inappropriate times. Verbal encouragement could be used more effectively. Score 1: Learner fails to use facilitative skills to encourage the patient to tell their story.""",
   "NON-VERBAL FACILITATION SKILLS": """Encouraging, supportive gestures, body language, and appropriate eye contact used; no physical barriers.
NOTE: Facilitative behavior is how comfortable the learner makes the patient feel with non-verbal communication. Non-verbal communication: appropriate non-verbal behavior, such as: eye contact, body language, facial expressions, reduction of physical barriers, physical contact. Learners should pay attention while avoiding staring at the patient or conducting the encounter as if it is an interrogation. Learners should not place a physical barrier between themselves and the patient. They should lean forward in a listening posture. Use of notes is their choice, some like to ensure accuracy by making notes. If they choose to read a chart, write notes, or use a computer, they do so in a manner that does not interfere with dialogue or rapport. Avoid crossing arms or leaning away, for instance, crossed arms during the sexual history suggests negative receptivity to the patient. Physical contact is sometimes appropriate.
Example: When a patient receives bad news or is upset, learners may show support by touching the patient’s hand or shoulder; they may also establish support by offering a tissue or drink of water.
Score 5: Puts patient at ease and facilitates communication by using good eye contact; relaxed, open body language; appropriate facial expression; eliminating physical barriers; and making appropriate physical contact with the patient. Score 3: Makes some use of facilitative techniques but could be more consistent. One or two techniques are not used effectively OR some physical barrier may be present. Score 1: Makes no attempt to put patient at ease. Body language is negative or closed OR any annoying mannerism (Example pencil tapping) intrudes on the interview. Eye contact not attempted or uncomfortable.""",
   "EMPATHY & ACKNOWLEDGING PATIENT CUES": """Empathetic approach, responds to concerns, helps to seek solutions.
NOTE: Empathy is not only being sensitive, but also demonstrating that sensitivity to patients so they appreciate the understanding and support. To display empathy, actively acknowledge and follow-up on verbal patient cues, showing that they have been heard and understood. The patient is actively encouraged to express emotion. Empathic statements are supportive comments that specifically link the “I” of the provider and the “you” of the patient. They both name and appreciate the patient’s affect or predicament and express appreciation for the problem.
“NURS” is an active technique to demonstrate empathy and acknowledge patient cues:
Naming emotion: “It must be very frustrating to not be able to work.”
Express Understanding [Goal is to normalize or validate feelings/experience]: "That must have been difficult for you. I’d feel that way too.”
Showing Respect: “I can appreciate how difficult it is for you to talk about this.”
Offering Support [Partnering/assistance, showing concern/sensitivity]: “I’ll be working with you each step of the way.”
Score 5: Learner uses supportive comments regarding patient’s emotions. Learner uses NURS (name, understand, respect, support) or specific techniques for demonstrating empathy. Score 3: Learner uses supportive comments inconsistently. Learner uses a few empathic statements; however, application in response to patient cues is inconsistent. Score 1: No empathy is demonstrated. Learner uses a negative emphasis or openly criticizes the patient.""",
   "ENCOURAGEMENT OF QUESTIONS": """Ask the patient if they have questions or additional concerns.
NOTE: Learners should encourage the patient to discuss additional points and ask questions by clearly providing an opportunity to do so. This can be done at the end of a major subsection and repeated at the end of the encounter.
Example: [near beginning]: “If you have questions at any time, feel free to ask.” [Then, as encounter progresses]: “Before we move on, any questions?”
Score 5: Learner encourages patient to ask questions and gives the opportunity to bring up additional topics or points not covered in the conversation. Score 3: Learner offers the opportunity to discuss any additional points or ask additional questions but neither encourages nor discourages the patient. Score 1: Learner fails to provide patient the opportunity to ask questions or discuss additional points. Learner may discourage questions.""",
   "ADMITTING LACK OF KNOWLEDGE": """When not equipped to provide specific information; learner admits it and offers to seek out the answer.
NOTE: Learners must be aware of their own level of experience as related to the information they are able to give to the patient. If asked for information/advice they are not equipped to provide, they admit lack of experience in that area, offer to seek resource(s) to answer the question and a timeframe is established for getting back to the patient with the answers.
Example: a provider referring a patient to a cardiologist may lack knowledge about specialized cardiovascular testing but should still help answer the patient concern/question.
Score 5: Learner, when asked for information or advice they are not equipped to provide, admits to a lack of knowledge in that area but immediately offers to seek resources to answer the question(s). Score 3: Learner, when asked for information or advice that they are not equipped to provide, admits lack of knowledge, but rarely seeks other resources for answers. Score 1: Learner, when asked for information, which they are not equipped to provide, makes up answers in an attempt to satisfy the patient’s questions, but never refers to other resources.""",
   "PATIENT EDUCATION & UNDERSTANDING": """Patient is given a comfortable amount of information; deliberate techniques to check understanding (have patient demonstrate/repeat the plan.)
NOTE: Many times, patients who are labeled non-compliant may in fact not understand the information that is given to them. There are several ways to check the patient's understanding. Learners can ask the patient to repeat the information directly back, demonstrate techniques, or pose hypothetical situations to see if the patient will react appropriately. It is vital that when a patient must carry out therapy on their own without direct supervision, that they understand how to carry it out.
Examples: Learner “I’ve shown you how to test levels of sugar in your blood with the monitor, now will you show me so I can be sure that I explained it clearly?” or “Will you repeat back to me how to take your medicine, so I know I have given you the correct information?”
Score 5: Learner uses deliberate techniques to check patient understanding of information given during the encounter including diagnosis. Techniques may include asking the patient to repeat information, asking if the patient has additional questions, posing hypothetical situations or asking the patient to demonstrate techniques. When patient education is a goal, learner determines the patient’s level of interest and provides education appropriately. (If English proficiency is limited an interpreter is offered.) Score 3: Learner asks the patient if they understand the information but does not use a deliberate technique to check. Some attempt to determine the interest in patient education but could be more thorough. Score 1: Learner fails to assess patient’s level of understanding and does not effectively correct misunderstandings when they are evident AND/OR Learner fails to address the issue of patient education.""",
   "ASSESS MOTIVATION FOR CHANGES": """Inquires about patient readiness for behavioral change
NOTE: The learner assesses how the patient feels about lifestyle and/or behavioral changes like taking medicine, changing diet and exercise, smoking cessation, etc. Many interviewers assume patients will change their behavior without discussing it with them. This lack of communication may lead to return visits or non-compliance issues. Asking the patient about previous experiences, the patient’s view of the importance to change, and the patient’s confidence in ability to change will help to establish guidelines. Then Learner can provide information, as appropriate, based on the patient’s needs. Offer a menu of options, emphasize the patient’s ability to choose, and anticipate and plan for obstacles.
STAGE  DESCRIPTION TECHNIQUES
Pre-contemplation  Not considering change  Identify patient’s goals, provide information, Bolster self-efficacy
Contemplation  Ambivalent to changing  Develop discrepancy between goals and behavior, Elicit self-motivational statements
Preparation  Cognitively committed to make the change  Strengthen commitment to change, Provide a menu of options for change
Action  Involved in change (began changing behaviors)  Identify new barriers, offer menu of options for reinforcing change
Maintenance  Involved in sustaining change (behavioral strategies are well learned and almost automatic)  Check status, Recognize relapse or impending relapse
Relapse  Undesired behavior returns  Identify relapse, reestablish self-efficacy and commitment to change, learn from experience, develop new behavioral strategy
Termination  Change is no longer an issue  None
Score 5: Learner inquires how the patient feels about the lifestyle/behavioral change and offers options and plans for the patient to choose from to encourage and/or support the change. Score 3: Learner inquires how the patient feels about changes but does not offer options or plans OR Learner offers options and plans but assumes the patient will follow the suggested change. Score 1: Learner fails to assess patient’s level of motivation to change and does not offer any options or plans.""",
   "INVESTIGATIONS AND PROCEDURES": """Discusses purpose of procedure/treatment, risks and benefits, and alternatives.
NOTE: If discussing investigations and procedures, learner provides clear information on procedures (What is going to be done), including what patient might experience (Will it hurt or harm? How much? How long?), and how patient will be informed of results (When and how will the patient be informed of results and the meaning of the results).
Learner relates procedures to treatment plan, value, and purpose; encourages discussions of potential anxieties or negative outcomes. Exploration of the patient questions, concerns, and clear explanations of procedures facilitate the likelihood of compliance.
Components of discussions include:
• Why the investigations & procedures must be done
• How the investigation/procedure is going to be done to include what the Patient might experience
• When and how the Patient will be informed of results and their meaning
• What potential anxieties or negative outcomes there may be.
Score 5: Learner discusses the purpose and nature, and reviews foreseeable risks and benefits of the proposed investigations or procedures, discloses alternative investigations or procedures and relative risks and benefits. Taking no action is considered an alternative. Score 3: Learner discusses some aspects of the investigations and procedures but omits some elements of informed consent. Score 1: Learner fails to discuss investigations or procedures.""",
   "ACHIEVE A SHARED PLAN": """Negotiates plan with patient and invites them to contribute ideas.
NOTE: A shared understanding is achieved with the patient, including nature and significance of the problem. The patient's understanding about the prognosis also plays a role in treatment
Example: someone who has had a family member die from a perforated ulcer may see a diagnosis of peptic ulcer as far more life threatening than the learner.
Learner involves the patient by making suggestions and encourages the patient to contribute thoughts, ideas, suggestions, and preferences. A mutually acceptable plan is negotiated, and learner checks in with patient on if the plan is acceptable and addresses concerns. Discussions between Learner and Patient include the: nature of the problem, significance of the problem and patient understanding about the problem/prognosis.
Examples: Learner - “It sounds like you would be interested in trying some medication to stop smoking. Of the options I told you about, which would you like to try first?” or “Since it is so hard for you to make physical therapy appointments, would it help if I suggest some ways to exercise your knee at work?”
Score 5: Learner discusses diagnosis and/or prognosis and negotiates a plan with the patient and invites the patient to contribute their own thoughts, ideas, suggestions, and preferences. Score 3: Learner discusses diagnosis and/or prognosis and plan but does not allow patient to contribute. Lacks full quality. Score 1: Learner fails to discuss diagnosis and/or prognosis.""",
   "CLOSURE": """Clearly specifies future plans (what learner will do, what patient should do, next communication date.)
NOTE: It is important that the patient feel that there is some closure at the end of the encounter. The patient must be left with a definite feeling about what will happen next, what Learner will do, what the patient should do, and when the next communication will occur. Closure will vary in detail according to the level of the Learner.
Effective closure to the healthcare visit contains three components:
•  What the Learner will do
•  What the Patient will do
•  When the next steps will occur
Example: “I will give you a prescription for antibiotics (what) and I would like you to get blood tests (what). I would like to see you again in one week (when).” Or “I will go speak to the team (what). If you would please change into your clothes (what). We will be back in a few minutes (when) to discuss your concerns together.”
Score 5: At the end of the encounter the learner clearly specifies the future plans: What learner will do (e.g., leave and consult, make referrals); What the patient will do (e.g., wait, make diet changes, and go to Physical Therapy); When (e.g., next communication or appointment will be.) Score 3: At the end of the encounter, the learner partially details the plans for the future. Score 1: At the end of the encounter, the learner fails to specify plans for the future and the patient leaves without a sense of what to expect. There is no closure whatsoever.""",
   "OVERALL INTERACTIVE TECHNIQUE": """Uses a patient-centered approach throughout the interview encounter.
NOTE: Use patient-centered interviewing techniques during the entire encounter. Patient-centered approach promotes a collaborative partnership between patient and Learner. However, patient-centered does not mean patient-controlled. A collaborative partnership promotes a more equal relationship, which enhances long-term compliance. Learner progresses from patient-centered to provider-centered technique to elicit required information but returns the lead to the patient whenever appropriate.
Score 5: Learner consistently uses the patient-centered techniques. Learner mixes patient-centered and provider-centered styles that promote a collaborative partnership between patient and the Learner. Score 3: Learner initially uses a patient-centered style but reverts to provider-centered at the end (rarely returns the lead to the patient) OR Learner uses all patient-centered and fails to use provider-centered style - therefore does not accomplish negotiated agenda. Score 1: Learner does not follow the patient’s lead. Uses only provider-centered technique halting the collaborative partnership.""",
   "ORGANIZATION": """The conversation/encounter follows a logical order; does not jump from section to section.
NOTE: Organization refers to the structure and organization of the entire conversation/encounter, encompassing the flow of the information gathered in the Introduction (when the Learner introduces self and explains their role), the body of the encounter, (Chief complaint and History of present illness, Past medical history, Family history, Social history, Review of systems), and the closure (the end of the encounter, but not the quality of the Closure because this is measured in another MIRS ITEM.)
Questions in the body of the encounter follow a logical order to the patient. Learner imposes structure by systematically following a series of topics.
Score 5: Questions in the body of the encounter follow a logical order to the patient. Score 3: Learner follows a series of topics or agenda items, but there are a few minor disjointed questions. Score 1: Learner asks questions that seem disjointed and unorganized.""",
   "PACING OF ENCOUNTER": """Neither too fast nor too slow with no interruptions or long pauses.
NOTE: Pacing refers to the rate of speech of the patient and the Learner. Pacing in the conversation does not refer to how quickly the content is obtained, but rather the tempo of the conversation process. Pacing should flow in a smooth and comfortable manner. Learners should not fire questions so fast that the patient has little or no time to consider their answers. Learners should be attentive to the patient and allow ample time to complete answers without interruption. Some delays are necessary [such as in reflective thinking] and are an indication of good interviewing skills. A period of silence may encourage the patient to provide additional relevant data or talk about sensitive issues they might otherwise omit.
Example, if patient behavior indicates a need to gather composure or ponder certain points, a delay can be beneficial.
Interruptions may be necessary when a patient moves off topic.
Example: The learner should politely stop the patient by saying something to the effect, “I’m sorry to interrupt, I’d like to ask you about…”
Score 5: Learner is attentive to the patient’s responses and listens without interruption. The conversation progresses smoothly with no awkward pauses. Silence may be used deliberately. Score 3: Learners’ pace is comfortable most of the time, the Learner occasionally interrupts patient and/or allows awkward pauses to break the flow of the conversation.
Score 1: Learner frequently interrupts patient, and there are awkward pauses, which break the flow of the conversation.
""",
   "TRANSITIONAL STATEMENTS OR “SIGNPOSTS”" : """Alert patient to change from one topic to another with reasons.
NOTE: Transitional statements are two-part statements used between subsections to inform the patient that a new topic is going to be discussed (what) and (why).
Example: “(What) I'd like to get some information about your past medical history, (Why) to see if it has any bearing on your present problem.”
With transitions, the patient understands why the learner is changing the subject and why they are seeking the information. Poor quality or no transitional statements can hinder the development of rapport and could result in a hostile or uncooperative patient. A transitional statement can be helpful when asking intimate or difficult questions such as sexual history.
Example: Learner - “I am now going to ask you some questions about your social history. Some of my questions may be sensitive in nature or embarrassing, but I am asking them to better care for you.”
""",
   "SUMMARIZATION" : """Data is summarized by the end of the interview or at end of each section.
NOTE: Summarization is a review of the main details elicited during the visit and is typically used at the end of each sub-section but may be used at any time. In summarizing the Chief Complaint and History of Present Illness it is important to provide a detailed summarization to the patient. In summarizing the Family History, a brief general statement may be sufficient, especially for a negative or non-complex positive family history. In Review of Systems, it is appropriate to summarize only the positives discovered.
Example: "Other than a few headaches you are fairly healthy. Our main task is to clear up the problem you're having with your back. Is this how you see the problem?"
Summarization of a body of information should be verified with the patient to make sure that all facts are correct. Verifying is done after summarization but may be utilized if a patient seems reluctant to interrupt, or to involve the patient in active listening.
Summarizing data at the end of each subsection serves several communication purposes:
• To "jog" the memory in case the learner has forgotten to ask a question.
• Allow the patient to hear how the learner understands the information.
• It proves to the patient that the learner has been listening, thus strengthening the relationship.
• Provides an opportunity to verify information:
Example: “To recap, your headache started yesterday, is on the left side, and is a pain level six. Is this correct?”
• Provides an opportunity to clarify information:
Example: "You said it is interfering with your attendance at school, how many days were missed?"
Score 5: Learner summarizes the data obtained at the end of each major line of inquiry or subsection to verify and/or clarify the information. Score 3: Learner summarizes the data at the end of some lines of inquiry, but not consistently or completely OR attempts to summarize at the end of the encounter and it is incomplete.
Score 1: Learner fails to summarize any of the data obtained."""
}


# ========= CATEGORY LABELS + ITEMS =========
CATEGORY_LABELS = {
    "OPEN": "Opens the Discussion",
    "GATH": "Gathers Information",
    "PERS": "Understands Patient Perspective",
    "SHARE": "Shares Information",
    "AGREE": "Reaches Agreement",
    "CLOSE": "Provides Closure",
    "REL": "Builds a Relationship",
}

CATEGORY_ITEMS = {
    "OPEN": [
      "OPENING",
      "ELICITS SPECTRUM OF CONCERNS",
      "NEGOTIATES PRIORITIES & SETS AGENDA",
    ],
    "GATH": [
      "ELICITING THE NARRATIVE THREAD",
      "TIMELINE",
      "TYPES OF QUESTIONS",
      "VERIFICATION OF PATIENT INFORMATION",
      "ORGANIZATION",
      "TRANSITIONAL STATEMENTS OR “SIGNPOSTS”",
      "SUMMARIZATION",
    ],
    "PERS": [
      "PATIENT'S PERSPECTIVE (BELIEFS)",
      "IMPACT OF ILLNESS ON PATIENT & SELF-IMAGE",
      "IMPACT OF ILLNESS ON FAMILY",
      "SUPPORT SYSTEMS",
    ],
    "SHARE": [
      "USE OF JARGON",
      "PATIENT EDUCATION & UNDERSTANDING",
      "INVESTIGATIONS AND PROCEDURES",
      "ADMITTING LACK OF KNOWLEDGE",
      "ENCOURAGEMENT OF QUESTIONS",
    ],
    "AGREE": [
      "ACHIEVE A SHARED PLAN",
      "ASSESS MOTIVATION FOR CHANGES",
    ],
    "CLOSE": [
      "CLOSURE",
      "SUMMARIZATION",
    ],
    "REL": [
      "OPENING",
      "VERBAL FACILITATION SKILLS & ENCOURAGEMENT",
      "NON-VERBAL FACILITATION SKILLS",
      "EMPATHY & ACKNOWLEDGING PATIENT CUES",
      "PACING OF ENCOUNTER",
    ],
}

def _xml_escape(s: str) -> str:
    s = s or ""
    return s.replace("&","&amp;").replace("<","&lt;").replace(">","&gt;")

def _history_as_xml(convo_history, max_chars=2400):
    """Emit Learner + Coach turns as XML, oldest→newest, trimmed by chars."""
    if not convo_history:
        return "<conversation_history />"

    turns = []
    total = 0
    # keep last ~40 turns; oldest→newest
    for m in convo_history[-40:]:
        role = (m.get("role") or "").lower()
        txt = (m.get("content") or "").strip()
        if not txt:
            continue
        tag_role = "learner" if role == "user" else ("coach" if role == "assistant" else "unknown")
        line = f'  <turn role="{tag_role}">{_xml_escape(txt)}</turn>'
        turns.append(line)
        total += len(txt)

    xml = "<conversation_history>\n" + "\n".join(turns) + "\n</conversation_history>"

    # Trim from the front if too long
    if len(xml) > max_chars:
        body = "\n".join(turns)
        keep = max_chars - len("<conversation_history>\n\n</conversation_history>")
        body = body[-keep:] if keep > 0 else ""
        xml = "<conversation_history>\n" + body + "\n</conversation_history>"

    return xml

def _examples_as_xml(examples) -> str:
    parts = []
    for ex in examples:
        inp = _xml_escape(ex["input"])
        cat = _xml_escape(ex["expect"]["category"])
        # keep reason in CDATA so we don't worry about punctuation/quotes
        reason = ex["expect"].get("reason", "")
        parts.append(
            "  <example>\n"
            f"    <input>{inp}</input>\n"
            f"    <expect category=\"{cat}\"><![CDATA[{reason}]]></expect>\n"
            "  </example>"
        )
    return "<examples>\n" + "\n".join(parts) + "\n</examples>"


def get_item_score(item: str, default=0.0):
    try:
        val = metrics.get(item, {}).get("score", default)
        return default if val is None else float(val)
    except Exception:
        return default

def format_item_block(category_key: str, max_chars_per_expl=None, divider=STAR_DIVIDER):
    lines = []
    for k in CATEGORY_ITEMS[category_key]:
        m = metrics.get(k, {})
        s = m.get("score", "NA")
        expl = (m.get("explanation") or "")
        if max_chars_per_expl is not None and len(expl) > max_chars_per_expl:
            expl = expl[:max_chars_per_expl].rstrip() + "…"
        lines.append(f"- {k}: {s}\n  why: {expl}")
    return divider.join(lines)

def _build_options_with_pretty_defs(divider: str):
    bare = divider.strip()  # "####"
    cats = {}
    for cid in CATEGORY_ITEMS:
        items = CATEGORY_ITEMS[cid]
        # build a single pretty block with separators around every item
        pretty_lines = []
        for it in items:
            definition = (MIRS_DEFINITIONS.get(it, "") or "").strip()
            pretty_lines.append(f"{bare} ITEM: {it}\n{definition}")
        cats[cid] = {
            "label": CATEGORY_LABELS[cid],
            "items": items,
            # This is what the model will see, with #### between each item
            "item_definition": "\n".join(pretty_lines),
        }
    return {
        "_divider": {"bare": bare, "with_newlines": divider},
        "categories": cats,
    }

def _flatten_history_for_classifier(history, max_chars=2400, only_user=True):
    """
    Turns your running chat 'history' (list of {'role','content'}) into a compact block.
    Most-recent context is most useful, so we keep the last ~40 turns and trim by chars.
    """
    if not history:
        return ""

    lines = []
    for m in history[-40:]:
        role = m.get("role", "")
        if only_user and role != "user":
            continue
        text = (m.get("content") or "").strip()
        if not text:
            continue
        who = "Learner" if role == "user" else "Coach"
        lines.append(f"{who}: {text}")

    block = "\n".join(lines)
    # keep the tail if it's too long (recent context > old context)
    if len(block) > max_chars:
        block = block[-max_chars:]
    return block

def category_score(category_key, method="mean"):
    vals = [get_item_score(k) for k in CATEGORY_ITEMS[category_key]]
    if not vals:
        return 0.0
    return (sum(vals)/len(vals)) if method == "mean" else min(vals)

def _norm(s: str) -> str:
    if s is None:
        return ""
    s = s.replace("\u2002", " ")
    s = s.replace("“", '"').replace("”", '"')
    s = s.replace("’", "'")
    s = re.sub(r"\s+", " ", s)
    return s.strip().lower()

_CLASSIFIER_STATE = {"last_category": None}
# ========= STRICT CLASSIFIER (unchanged logic from your fixed version) =========
def map_mirs_category(learner_input: str, client: OpenAI, convo_history=None, max_history_chars=2400, _state=_CLASSIFIER_STATE):
    """
    Returns (category_id, reason). NEVER returns confidence.
    Now considers the WHOLE conversation via convo_history.
    """
    allowed_ids = list(CATEGORY_ITEMS.keys()) + ["UNMAPPABLE"]

    # Build category/options payload once (pretty defs with divider already in code)
    history_block = _flatten_history_for_classifier(convo_history, max_chars=max_history_chars, only_user=True)
    history_xml = _history_as_xml(convo_history or [], max_chars=max_history_chars)
    options_for_prompt = _build_options_with_pretty_defs(STAR_DIVIDER)
    # when building options_for_prompt, also prepare:
    options_xml = "\n".join(
    f'''  <category id="{cid}" label="{_xml_escape(CATEGORY_LABELS[cid])}">
    <items>{''.join(f"<item>{_xml_escape(it)}</item>" for it in CATEGORY_ITEMS[cid])}</items>
    <definitions><![CDATA[
    {_build_options_with_pretty_defs(STAR_DIVIDER)['categories'][cid]['item_definition']}
    ]]></definitions>
    </category>'''
    for cid in CATEGORY_ITEMS
    )

    prev_hint = _state.get("last_category")
    prev_hint = prev_hint if prev_hint in CATEGORY_ITEMS else "(none)"
    # Few-shot examples (deduped; keep concise)
    examples = [
        {"input": "Gathers Information", "expect": {"category":"GATH","reason":"direct category mention"}},
        {"input": "Opens the Discussion", "expect": {"category":"OPEN","reason":"category label"}},
        {"input": "Opening", "expect": {"category":"REL","reason":"item 'OPENING' lives under REL"}},
        {"input": "Eliciting the Narrative Thread", "expect":{"category":"GATH","reason":"narrative thread now grouped under GATH in your map"}},
        {"input": "Timeline", "expect":{"category":"GATH","reason":"timeline"}},
        {"input": "I used plain language and teach-back.", "expect":{"category":"SHARE","reason":"jargon/teach-back"}},
        {"input": "We chose the plan together.", "expect":{"category":"AGREE","reason":"shared plan"}},
        {"input": "I summarized and set next steps.", "expect":{"category":"CLOSE","reason":"summary/plan"}},
        {"input": "The beginning part", "expect":{"category":"OPEN","reason":"vague opening, default to OPEN"}},
        {"input": "I mostly want feedback on how fast I typed.", "expect":{"category":"UNMAPPABLE","reason":"not a MIRS skill"}},
    ]
    examples_xml = _examples_as_xml(examples)
    output_format_block = '{{"category":"<one of {ids}>","reason":"<brief>"}}'.format(ids=allowed_ids)


    system_prompt = f"""
You are a STRICT classifier for clinical interviewing/communication skills (MIRS).
<response_format_hint>json</response_format_hint>
<config>
  <previous_category>{_xml_escape(prev_hint)}</previous_category>
  <history_window_chars>{max_history_chars}</history_window_chars>
</config>

<allowed_ids>
  {''.join(f'<id>{cid}</id>' for cid in allowed_ids)}
</allowed_ids>

<policy>
  <minimize_switching default_to="previous_category" />
  <switch_triggers>
    <trigger>Exact category label (case-insensitive) not inside quotes</trigger>
    <trigger>Exact MIRS item name (not inside quotes)</trigger>
    <trigger>Canonical synonyms (teach-back→SHARE, anything else?→OPEN, timeline/chronology→GATH, shared plan→AGREE, summarize/next steps→CLOSE, NURS/empathy→REL)</trigger>
    <trigger>Explicit contrast: "not X, I want Y"</trigger>
  </switch_triggers>
  <non_triggers>
    <non_trigger>Label or item only inside quotes</non_trigger>
    <non_trigger>Generic verbs/structure words (start, move on, also, flow)</non_trigger>
    <non_trigger>Meta talk without item label (transitions, pacing, signposting)</non_trigger>
    <non_trigger>Polite fillers/affirmations</non_trigger>
    <non_trigger>Older history contradicting latest Learner turn</non_trigger>
  </non_triggers>
  <decision_order>
    <step>Use a switch trigger if present; explain briefly in reason</step>
    <step>If clear mapping without trigger, choose that category</step>
    <step>Else keep previous_category and say "kept previous due to ambiguity"</step>
  </decision_order>
  <category_bias>
    If an item under OPEN/GATH/PERS/SHARE/AGREE/CLOSE is named/implied, choose it (NOT REL).
    Disambiguation: "Opens the Discussion" = OPEN label; "Opening" = REL item 'OPENING'.
  </category_bias>
</policy>

<options>
  {options_xml}
</options>

<conversation_history>
  {history_xml}
</conversation_history>

<input_text>{_xml_escape(learner_input)}</input_text>

<examples>
  {examples_xml}
</examples>

<output_format>
{output_format_block}
</output_format>

"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"INPUT_TEXT:\n{learner_input}"}
    ]

    try:
        resp = client.chat.completions.create(
            model="gpt-4.1-2025-04-14",
            messages=messages,
            temperature=0,
            max_tokens=120,
            response_format={"type": "json_object"},
        )
        raw = (resp.choices[0].message.content or "").strip()
        data = json.loads(raw)
        cat = (data.get("category") or "").strip()
        reason = (data.get("reason") or "").strip()

        if cat in CATEGORY_ITEMS:
          _state["last_category"] = cat
        if cat in CATEGORY_ITEMS or cat == "UNMAPPABLE":
            return cat, (reason or "model classification")
        return "UNMAPPABLE", "unknown category"
    except Exception as e:
        print("map_mirs_category error:", e)
        return "UNMAPPABLE", "classifier error"

# ========= Tone detector (unchanged) =========
def detect_emotion_from_input(learner_input: str):
    system_prompt = (
        "You're an expert at detecting tone. "
        "Label the user's emotional state as one of: "
        "['confident','uncertain','frustrated','neutral','disappointed','sad','proud'].\n"
        "Return just one word."
    )
    user_prompt = f'User reflection: """{learner_input}"""\nTone:'
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=[
                {"role":"system","content":system_prompt},
                {"role":"user","content":user_prompt}
            ],
            temperature=0,
            max_tokens=5
        )
        return response.choices[0].message.content.strip().lower()
    except Exception as e:
        print("Emotion detection error:", e)
        return "neutral"

# ========= Helpers for definitions =========
def category_definitions_block(category_key, divider=STAR_DIVIDER):
    parts = []
    for item in CATEGORY_ITEMS[category_key]:
        parts.append(f"### {item}\n{MIRS_DEFINITIONS[item].strip()}")
    return divider.join(parts)

# ========= fixed coach identity + dynamic category slice =========
COACH_IDENTITY_XML = f"""
You are an **Expert MIRS Communication Coach**.
Your job is to guide medical students in reflecting on and improving their communication skills, using the Medical Interview Rating Scale (MIRS) as the framework.
#########################
<context>
- MIRS evaluates communication dimensions (scored from 0-5).
- The active category is supplied by the system each turn and may change automatically between turns.
- Do **not** ask the learner to confirm a switch and do **not** announce a switch.
- Inputs are listed at the bottom of the system prompt:
 1) CATEGORY: Focus category for this turn (REL/OPEN/GATH/PERS/SHARE/AGREE/CLOSE). Coach only within this category; don’t announce switches.
 2) ITEMS IN SCOPE: The item IDs within CATEGORY. Cite only these.
 3) SCORES (this category): Per-item score + brief rationale. Use as evidence; do not re-grade.
 4) DEFINITIONS: Authoritative item definitions. Tie all feedback/goals to these behaviors.
</context>
#########################
<task>
- Focus only on the specified category
- Address the learner’s input
- Across the dialogue, aim to move flexibly through reflection → feedback → goals. In each single response, focus on just one of these steps.
</task>
########################
<strict_rules>
• Coach on THIS category only
• Be an approachable, professional coach who supports the student’s learning agenda, not a lecturer.
• Stay within MIRS communication skills only.
• After offering feedback or a goal, invite the student to rephrase or adjust it so it feels like theirs.
• Keep replies brief (3–6 sentences) and ask only one guiding question.
• Invite clarification and offer examples/scaffolds when helpful.
• Quote or reference the learner’s inputs when possible.
• Encourage growth sincerely.
• Correct the user if they say anything inacurate. Push back on any incorrect assessments or quotes—kindly.
• Use a warm and human tone. Encourage the learner whenever you can.
</strict_rules>
"""

def category_context(category_key: str) -> str:
    #print(category_key)
    defs_block = category_definitions_block(category_key)
    item_scores = format_item_block(category_key, max_chars_per_expl=320)
    return f"""
############################################################
CATEGORY: {CATEGORY_LABELS.get(category_key, category_key)}
############################################################
ITEMS IN SCOPE: {', '.join(CATEGORY_ITEMS[category_key])}
############################################################
SCORES (this category)
{item_scores}
############################################################
DEFINITIONS (this category)
{defs_block}
""".strip()

def build_transcript_block():
    """Convert transcript to Clinician/Patient only."""
    transcript_block = "\n".join(f"{m['role'].title()}: {m['content']}" for m in transcript)
    filtered = []
    for line in transcript_block.splitlines():
        t = line.strip()
        if t.startswith("User:") or t.startswith("Assistant:"):
            filtered.append(t)
    return "\n".join(filtered).replace("User:", "Clinician:").replace("Assistant:", "Patient:")
def build_transcript_xml():
    turns = []
    # Use your loaded `transcript` list (from the CSV) like you already do
    for m in transcript:
        role = (m.get("role") or "").lower()
        content = _xml_escape(m.get("content") or "")
        if role == "user":
            turns.append(f'  <turn role="clinician">{content}</turn>')
        elif role == "assistant":
            turns.append(f'  <turn role="patient">{content}</turn>')
    body = "\n".join(turns) if turns else ""
    return "<transcript>\n" + body + "\n</transcript>"

def _item_scores_xml(category_key: str) -> str:
    rows = []
    for it in CATEGORY_ITEMS[category_key]:
        m = metrics.get(it, {})
        score = m.get("score", "NA")
        why = (m.get("explanation") or "").strip()
        rows.append(
            f'    <item id="{_xml_escape(it)}">'
            f"<score>{_xml_escape(str(score))}</score>"
            f"<why>{_xml_escape(why)}</why>"
            f"</item>"
        )
    return "<scores>\n" + "\n".join(rows) + "\n</scores>"

def _items_in_scope_xml(category_key: str) -> str:
    items = "\n".join(f"    <item>{_xml_escape(it)}</item>" for it in CATEGORY_ITEMS[category_key])
    return "<items_in_scope>\n" + items + "\n</items_in_scope>"

def category_definitions_xml(category_key: str, divider=STAR_DIVIDER) -> str:
    # reuse human-readable block but wrap in CDATA so we don't have to escape it
    text_block = category_definitions_block(category_key, divider=divider)
    return "<definitions><![CDATA[\n" + text_block + "\n]]></definitions>"

HASH_BAR = "\n" + ("#" * 15) + "\n"
def category_context_xml(category_key: str, divider: str = HASH_BAR) -> str:
    cid = category_key
    label = CATEGORY_LABELS.get(cid, cid)
    div = _xml_escape(divider)
    return (
        "<category_context>\n"
        f"{div}"
        f'  <category id="{_xml_escape(cid)}" label="{_xml_escape(label)}" />\n'
        f"{div}"
        f"  {_items_in_scope_xml(cid)}\n"
        f"{div}"
        f"  {_item_scores_xml(cid)}\n"
        f"{div}"
        f"  {category_definitions_xml(cid)}\n"
        f"{div}"
        "</category_context>"
    )

# ========= Coach that rebuilds ONLY the category slice each turn =========
def chatbot_category(score_dict, learner_input: str, category_key: str, history=None, debug=False):
    if history is None:
        history = []

    _ = detect_emotion_from_input(learner_input or "")  # keep tone hook; not used directly
    clean_transcript = build_transcript_block()

    # One system message per call: fixed identity + dynamic category context + static transcript
    transcript_xml = build_transcript_xml()
    #print(transcript_xml)
    system_prompt = (
    COACH_IDENTITY_XML
    + "\n\n"
    + category_context_xml(category_key)
    + "\n\n"
    + transcript_xml
)

    messages = [{"role":"system","content":system_prompt}] + history[:]
    if learner_input:
        messages.append({"role":"user","content":learner_input})

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-2025-04-14",
            messages=messages,
            temperature=0.7,
            max_tokens=650
        )
        reply = response.choices[0].message.content
        if learner_input:
            history.append({"role":"user","content":learner_input})
        history.append({"role":"assistant","content":reply})
        return reply, history
    except Exception as e:
        print("GPT error:", e)
        return "⚠", history

# ========= Utilities to find a category from text =========
def category_for_item(item_key):
    for cat, items in CATEGORY_ITEMS.items():
        if item_key in items:
            return cat
    return None

def ask_until_mappable():
    """
    Keep prompting until we can map to a MIRS category.
    Returns (category_key, user_text) or (None, None) if 'exit'.
    """
    while True:
        first = input(
          "What's the ONE thing you most want feedback on?\n"
          "(for example: 'opening the discussion', 'gathering information', 'understanding the paient perspective', 'sharing information', 'reaching agreement', 'providing closure', 'building a relationship', etc. Type 'exit' to quit.)\nLearner: "
        ).strip()
        if first.lower() in {"exit","quit"}:
            return None, None

        # Exact matches
        norm = first.lower().strip()
        for c in CATEGORY_ITEMS.keys():
            if c.lower().strip() == norm:
                return c, first
        for it in {x for xs in CATEGORY_ITEMS.values() for x in xs}:
            if it.lower() == norm:
                mapped = category_for_item(it)
                if mapped:
                    return mapped, first

        # LLM map
        category_key, reason = map_mirs_category(first, client)
        print("↳ classifier:", category_key, "|", reason)
        if category_key != "UNMAPPABLE":
            return category_key, first
        print("\nI couldn't map that to a MIRS category. Try a specific skill term or a category name (e.g., OPEN, GATH, REL).\n")
def ask_next_interview_goal():
    """
    Ask once at the end of the encounter.
    Returns a trimmed string or None if skipped.
    """
    print("\n— Before we wrap —")
    goal = input("What are your goals for your next interview? (Press Enter to skip)\nLearner: ").strip()
    return goal if goal else None

# ========= Session runner (only change: reclassify each turn; rebuild category slice) =========
def run_chatbot_category(score_dict, debug_routing=False, hard_switch_on_category_change=True):
    history = []

    # First focus
    cat, first_text = ask_until_mappable()
    if cat is None:
        #print("Ended.")
        return history

    current_category = cat
    #if debug_routing:
        #print(f"→ start category: {current_category} ({CATEGORY_LABELS.get(current_category, current_category)})")

    # First coached reply
    out, history = chatbot_category(
        score_dict,
        learner_input=first_text,
        category_key=current_category,
        history=history,
        debug=False,
    )
    print("\nDialogic Feedback Coach:\n", out)

    # Ongoing loop with reclassification each turn
    while True:
        cont = input("\n Your next thought (Enter to finish):\nLearner: ").strip()
        if cont == "":
            break

        # inside run_chatbot_category loop:
        new_cat, reason = map_mirs_category(cont, client, convo_history=history)

        if debug_routing:
            print(f"→ classifier: {new_cat} ({CATEGORY_LABELS.get(new_cat, new_cat)}) | reason: {reason}")

        if new_cat != "UNMAPPABLE" and new_cat != current_category:
            current_category = new_cat
            if hard_switch_on_category_change:
                # Hard switch: clear prior convo so the new category context fully dominates
                history = []
            else:
                # Soft switch: keep continuity but announce the focus change
                history.append({
                    "role": "assistant",
                    "content": f"(Switching focus to **{CATEGORY_LABELS.get(current_category, current_category)}** to match your question.)"
                })
            if debug_routing:
                print(f"→ switched to: {current_category} ({CATEGORY_LABELS.get(current_category, current_category)})")

        out, history = chatbot_category(
            score_dict,
            learner_input=cont,
            category_key=current_category,
            history=history,
            debug=False,
        )
        print("\n Dialogic Feedback Coach:\n", out)

    # Add the goal prompt here, just before returning
    next_goal = ask_next_interview_goal()
    #if next_goal:
      #  print("\nSaved goal for your next interview:", next_goal)

    return history



def run_full_session():
    return run_chatbot_category(score_dict)

# Kick off
run_full_session()


What's the ONE thing you most want feedback on?
(for example: 'opening the discussion', 'gathering information', 'understanding the paient perspective', 'sharing information', 'reaching agreement', 'providing closure', 'building a relationship', etc. Type 'exit' to quit.)
Learner: opening

Dialogic Feedback Coach:
 Let’s reflect on how you opened the encounter. You greeted the patient by name and asked about pronouns (“Can I just confirm what pronouns you use?”), and you checked how to address her (“is it okay if I refer to you as Miss Bishop?”). This shows thoughtfulness and respect.

However, the MIRS definition for OPENING expects you to also clarify your role (for example, “I’m a medical student working with your care team today”) and ideally to use the patient’s name directly in your introduction. These elements help set expectations and build rapport right from the start.

Looking back, how did you feel about the way you introduced yourself? Is there anything you wish you had inc

KeyboardInterrupt: Interrupted by user